# Labor Condition Applications (LCAs)
## Combine data across years and deduplicate

Combine data for all years and deduplicate so that there is only one record per case number.

Standardize employer names
Add additional fields

In [1]:
# Import packages
import pandas as pd

In [2]:
# Set up parameters
data_dir = '../../data/'
input_dir = data_dir + 'raw/'
input_filename_base = 'lca_raw_'
output_dir = data_dir + 'intermediate/'
output_filename = 'lca_deduped.csv'

years = list(range(2010, 2023+1))

### Load data across all years

In [3]:
# Load data from the individual year files into a single dataframe
lca = []
for year in years:
  lca.append(pd.read_csv(input_dir + input_filename_base + str(year) + '.csv', dtype=str))
lca = pd.concat(lca)

### Keep only certified cases
The final dataset will contain only certified petitions (CASE_STATUS == "Certified") and only one record per case (i.e. CASE_NUMBER is unique key)

In [4]:
lca['CASE_STATUS'].value_counts(dropna=False)

CASE_STATUS
CERTIFIED                                             4595557
Certified                                             2482470
CERTIFIED-WITHDRAWN                                    372118
WITHDRAWN                                              163560
DENIED                                                 158085
Certified - Withdrawn                                  127650
Withdrawn                                               49746
Denied                                                  14464
PENDING QUALITY AND COMPLIANCE REVIEW - UNASSIGNED         15
REJECTED                                                    2
INVALIDATED                                                 1
Name: count, dtype: int64

Drop all records of cases that have a status of "Denied", "Withdrawn", "Certified - Withdrawn", or anything other than "Certified"

In [5]:
drop_cases = lca[lca['CASE_STATUS'].str.lower() != 'certified']['CASE_NUMBER'].tolist()
lca = lca[~lca['CASE_NUMBER'].isin(drop_cases)]

Check that only certified cases are left

In [6]:
lca.groupby('CASE_STATUS').agg(n=('CASE_STATUS', 'count'))

,n
CASE_STATUS,
CERTIFIED,4403458
Certified,2407200


### Deduplicate case numbers
We want our final dataset to be unique by case number.

Confirm that there are duplicate cases. This is expected since we imported all LCA files with the intention of deduplicating later.

In [7]:
dupe_case_numbers = lca.groupby('CASE_NUMBER').agg(n=('CASE_NUMBER', 'count'))
dupe_case_numbers = dupe_case_numbers[dupe_case_numbers['n'] > 1]
dupe_case_numbers

,n
CASE_NUMBER,
I-200-13319-873657,2
I-200-15037-909445,2
I-200-15041-483351,2
I-200-15047-012762,2
I-200-15047-038903,2
...,...
I-203-22356-657976,2
I-203-22356-657986,2
I-203-22356-658202,2


We'll separate the unique and duplicate cases into multiple dataframes to make it easier to work with

In [8]:
duplicate_case_filter = lca['CASE_NUMBER'].isin(dupe_case_numbers.index.tolist())

In [9]:
# Dataframe with duplicate case numbers
lca_dupes = lca.copy().loc[duplicate_case_filter, :]
print(str(lca_dupes.shape[0]) + ' rows in duplicate cases dataframe: ')

623614 rows in duplicate cases dataframe: 


In [10]:
# Dataframe with unique cases
lca = lca.copy().loc[~duplicate_case_filter, :]
print('Rows in unique cases dataframe: ' + str(lca.shape[0]))

Rows in unique cases dataframe: 6187044


See how duplicate cases are distributed across years

In [11]:
lca_dupes.groupby('DATAFILE_YEAR').agg(n=('CASE_NUMBER', 'count'))

,n
DATAFILE_YEAR,
2014,1
2015,147
2016,1
2021,450551
2023,172914


Most duplicate cases are in 2021 and 2023. Data for these years was pulled from quarterly files which the DOL website says are cumulative. In this case, if the case appears in multiple quarter files, keep the record from the most recent quarter.

Pre-2020 data are contained in a single yearly file. Duplicates for these years are minimal. We'll handle these later.

In [12]:
# # Create a combined year-quarter field to help deduplicate
# def create_year_quarter(row):
#   if pd.isna(row['DATAFILE_QUARTER']):
#     val = str(row['DATAFILE_YEAR'])
#   else:
#     val = str(row['DATAFILE_YEAR']) + '-Q' + str(int(row['DATAFILE_QUARTER']))
#   return val

# lca['YEAR_QUARTER'] = lca.apply(create_year_quarter, axis=1)

lca_dupes['YEAR_QUARTER'] = lca_dupes['DATAFILE_YEAR'] + '-Q' + lca_dupes['DATAFILE_QUARTER'].fillna('')

In [13]:
# Check the distribution of YEAR_QUARTER
# Note that pre-2020 data will not have a quarter
lca_dupes['YEAR_QUARTER'].value_counts(dropna=False).sort_index()

YEAR_QUARTER
2014-Q          1
2015-Q        147
2016-Q          1
2021-Q1     70429
2021-Q2    190061
2021-Q3    190061
2023-Q1     86457
2023-Q2     86457
Name: count, dtype: int64

In [14]:
# For cases with multiple records, find the most recent decision date from the most recent data file
lca_keep = lca_dupes.groupby('CASE_NUMBER').agg({'YEAR_QUARTER': 'max'}).reset_index()

In [15]:
# Join on the original dataset to get the rest of the fields
lca_join_keys = ['CASE_NUMBER', 'YEAR_QUARTER']
lca_dupes = (
  lca_dupes
    .set_index(lca_join_keys)
    .join(
      lca_keep.set_index(lca_join_keys),
      how = 'inner',
      lsuffix='_LEFT',
      rsuffix='_RIGHT'
    )
    .reset_index()
)

Check again for duplicates

In [16]:
dupe_case_numbers = lca_dupes.groupby('CASE_NUMBER').agg(n=('CASE_NUMBER', 'count'))
dupe_case_numbers = dupe_case_numbers[dupe_case_numbers['n'] > 1]
duplicate_case_filter = lca_dupes['CASE_NUMBER'].isin(dupe_case_numbers.index.tolist())

lca_dupes[duplicate_case_filter].groupby('DATAFILE_YEAR').agg(n=('CASE_NUMBER', 'count'))

,n
DATAFILE_YEAR,
2015,147


The rest of the duplicates are from 2015.  
Let's check if if these cases have multiple decision dates. If so, we'll keep the most record with the most recent decision date.

In [17]:
decision_dates = (
  lca_dupes[duplicate_case_filter]
    .sort_values(['CASE_NUMBER', 'DECISION_DATE'], ascending=False)
    .loc[:, ['CASE_NUMBER', 'DECISION_DATE']]
    .groupby('CASE_NUMBER')['DECISION_DATE']
    .apply(list)
    .reset_index(name='DECISION_DATE')
)
decision_dates['n_decision_dates'] = decision_dates.apply(lambda row: len(set(row['DECISION_DATE'])), axis=1)

print(str(decision_dates[decision_dates['n_decision_dates'] > 1].shape[0]) + ' records with multiple decision dates')


54 records with multiple decision dates


Around a third have multiple decision dates. We'll keep the records with the most recent one.

In [18]:
lca_keep = lca_dupes.groupby('CASE_NUMBER').agg({'DECISION_DATE': 'max'}).reset_index()

In [19]:
lca_join_keys = ['CASE_NUMBER', 'DECISION_DATE']
lca_dupes = (
  lca_dupes
    .set_index(lca_join_keys)
    .join(
      lca_keep.set_index(lca_join_keys),
      how = 'inner',
      lsuffix='_LEFT',
      rsuffix='_RIGHT'
    )
    .reset_index()
)

Check the duplicates count again

In [20]:
dupe_case_numbers = lca_dupes.groupby('CASE_NUMBER').agg(n=('CASE_NUMBER', 'count'))
dupe_case_numbers = dupe_case_numbers[dupe_case_numbers['n'] > 1]
duplicate_case_filter = lca_dupes['CASE_NUMBER'].isin(dupe_case_numbers.index.tolist())

lca_dupes[duplicate_case_filter].groupby('DATAFILE_YEAR').agg(n=('CASE_NUMBER', 'count'))

,n
DATAFILE_YEAR,
2015,27


A handful left, all from 2015. Let's have a look.

In [21]:
lca_dupes[duplicate_case_filter].sort_values(['CASE_NUMBER'])

,CASE_NUMBER,DECISION_DATE,YEAR_QUARTER,CASE_STATUS,RECEIVED_DATE,BEGIN_DATE,END_DATE,EMPLOYER_NAME,EMPLOYER_CITY,EMPLOYER_STATE,...,WORKSITE_WORKERS,SECONDARY_ENTITY,SECONDARY_ENTITY_BUSINESS_NAME,PW_TRACKING_NUMBER,PW_OES_YEAR,PW_SURVEY_PUBLISHER,PW_SURVEY_NAME,PW_OTHER_YEAR,TOTAL_WORKSITE_LOCATIONS,STATUTORY_BASIS
1,I-200-15037-909445,2015-03-05 00:00:00,2015-Q,CERTIFIED,2015-02-24 00:00:00,08/12/2015,08/11/2018,TRADEWEB MARKETS LLC,NEW YORK,NY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,I-200-15037-909445,2015-03-05 00:00:00,2015-Q,CERTIFIED,2015-02-24 00:00:00,08/12/2015,08/11/2018,"ACTIMIZE, INC.",NEW YORK,NY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,I-200-15048-295792,2015-02-23 00:00:00,2015-Q,CERTIFIED,2015-02-17 00:00:00,08/14/2015,08/12/2018,INFOSYS LIMITED,PLANO,TX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,I-200-15048-295792,2015-02-23 00:00:00,2015-Q,CERTIFIED,2015-02-17 00:00:00,08/14/2015,08/12/2018,INFOSYS LIMITED,PLANO,TX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,I-200-15048-627298,2015-02-23 00:00:00,2015-Q,CERTIFIED,2015-02-17 00:00:00,08/14/2015,08/12/2018,INFOSYS LIMITED,PLANO,TX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,I-200-15048-627298,2015-02-23 00:00:00,2015-Q,CERTIFIED,2015-02-17 00:00:00,08/14/2015,08/12/2018,INFOSYS LIMITED,PLANO,TX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,I-200-15048-693795,2015-02-23 00:00:00,2015-Q,CERTIFIED,2015-02-17 00:00:00,08/14/2015,08/12/2018,INFOSYS LIMITED,PLANO,TX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,I-200-15048-693795,2015-02-23 00:00:00,2015-Q,CERTIFIED,2015-02-17 00:00:00,08/14/2015,08/12/2018,INFOSYS LIMITED,PLANO,TX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,I-200-15071-802969,2015-03-19 00:00:00,2015-Q,CERTIFIED,2015-03-13 00:00:00,03/19/2015,03/18/2018,"TECHTU BUSINESS SOLUTIONS, INC.",PLEASANTON,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,I-200-15071-802969,2015-03-19 00:00:00,2015-Q,CERTIFIED,2015-03-13 00:00:00,03/19/2015,03/18/2018,"TECHTU BUSINESS SOLUTIONS, INC.",PLEASANTON,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Manually inspecting the data, it looks like there one or more variations in employer name, job title, and wage rate of pay within the same case number.  
It's not clear what's the best way to resolve these issues. Are these actually separate cases/applications that where somehow assigned the same case number? Do they represent modifications or updates to an existing case but were recorded twice?  

We could retain all these cases. To do this, we could modify the case number to make it unique (e.g., adding "-1" and "-2" to the end of case numbers). But this would make it more difficult to match the case back to the original dataset.  
We could also create a new field that indicates duplicates, but this would mean case numbers would still not be unique and could not be used as an index.  

Given that there are so few of these - fewer than 30 cases in a file of 4.4M cases, and representing ~200 worker positions - the easiest thing may be to drop these records since it would have minimal impact on aggregate metrics.  


In [22]:
drop_cases = lca_dupes[duplicate_case_filter]['CASE_NUMBER'].tolist()
lca_deduped = lca_dupes[~lca_dupes['CASE_NUMBER'].isin(drop_cases)]

Confirm that there are no duplicates in deduped dataset before adding it to the rest of the cases.

In [28]:
lca_deduped.groupby('CASE_NUMBER').agg(n=('CASE_NUMBER', 'count')).sort_values('n', ascending=False)

,n
CASE_NUMBER,
I-200-13319-873657,1
I-200-21083-172229,1
I-200-21083-172205,1
I-200-21083-172206,1
I-200-21083-172207,1
...,...
I-200-21022-027075,1
I-200-21022-027085,1
I-200-21022-027088,1


No duplicates. Add these records to the main dataset.

In [24]:
print('Unique records: ' + str(lca.shape[0]))
print('Deduped records: ' + str(lca_deduped.shape[0]))
print('Final record number: ' + str(lca.shape[0] + lca_deduped.shape[0]))

Unique records: 6187044
Deduped records: 276572
Final record number: 6463616


In [33]:
lca = pd.concat([lca, lca_deduped], ignore_index=True)

Check record count

In [34]:
lca.shape

(6463616, 54)

Check that there are no duplicates

In [35]:
lca.groupby('CASE_NUMBER').agg(n=('CASE_NUMBER', 'count')).sort_values('n', ascending=False)

,n
CASE_NUMBER,
I-200-09105-013231,1
I-200-19259-737552,1
I-200-19259-733634,1
I-200-19259-732384,1
I-200-19259-732033,1
...,...
I-200-15274-248348,1
I-200-15274-246657,1
I-200-15274-244784,1


### Save the file

In [36]:
# Save the dataset
lca.to_csv(output_dir + output_filename, index=False)